In [1]:
import sys
sys.path.append(r'D:\CE7454_2018\VAE FOR STOCHASTIC INTEGER PROGRAMMING\CVAE for stochastic programming')
from local_stochoptim.stochprob.network_design import network_design_problem as ndp

In [2]:
import numpy as np
#import .local.stochoptim.stochprob.network_design.network_design_problem as ndp
import stochoptim.stochprob.network_design.network_design_solution as nds
import stochoptim.stochprob.network_design.network_design_uncertainty as ndu
from stochoptim.scenclust.cost_space_partition import CostSpaceScenarioPartitioning
import stochoptim.scengen.scenario_tree as st
np.set_printoptions(linewidth=120)

In [3]:
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset, TensorDataset
from collections import defaultdict

# for clustering
from sklearn_extra.cluster import KMedoids
#from pyclustering.cluster.kmedoids import kmedoids
import pickle

### Generate Problem Settings

In [4]:
n_origins = 2
n_destinations = 2
n_intermediates = 10
#=======================
n_scenarios = 200
distribution = 'uniform'
lb = 5
ub = 20
corr = 0.4

### No objectives

In [5]:
# from models import VAE
# torch.manual_seed(0)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # the net
# vae = VAE(
#     encoder_layer_sizes=[4, 256],
#     latent_size=2,
#     decoder_layer_sizes=[256, 4],
#     conditional=True,
#     num_labels=2 if True else 0).to(device)

# # Specify a path
# PATH = "trained_nets/saved_model_"+ str(1999) + ".pt"
# # load parameters
# vae.load_state_dict(torch.load(PATH))
# vae.eval()
############################################################GCN
from config import *
config_path = "configs/tsp10.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
from models_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae = VAE(
    config=config,     
    encoder_layer_sizes=[4, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 4],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/saved_model_GCN_"+ str(100) + ".pt"
# load parameters
vae.load_state_dict(torch.load(PATH))
vae.eval()

Loaded configs/tsp10.json:
{'expt_name': 'tsp10', 'gpu_id': '0', 'train_filepath': './data/tsp10_train_concorde.txt', 'val_filepath': './data/tsp10_val_concorde.txt', 'test_filepath': './data/tsp10_test_concorde.txt', 'num_nodes': 14, 'num_neighbors': -1, 'node_dim': 2, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 30, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=4, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=3, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=128, out_features=128, bias=True)
          (V): Linear(in_features=128, out_features=128, bias=True)
        )
        (edge_feat): EdgeFeatures(
          (

### With objectives

In [6]:
# from models_objs import VAE
# torch.manual_seed(0)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # the net
# vae_objs = VAE(
#     encoder_layer_sizes=[4, 256],
#     latent_size=2,
#     decoder_layer_sizes=[256, 4],
#     conditional=True,
#     num_labels=2 if True else 0).to(device)

# # Specify a path
# PATH = "trained_nets/ob_saved_model_"+ str(1999) + ".pt"
# # load parameters
# vae_objs.load_state_dict(torch.load(PATH))
# vae_objs.eval()
################################################### GCN
from config import *
config_path = "configs/tsp10.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
from models_objs_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae_objs = VAE(
    config=config,     
    encoder_layer_sizes=[4, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 4],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/ob_saved_model_GCN_"+ str(300) + ".pt"
# load parameters
vae_objs.load_state_dict(torch.load(PATH))
vae_objs.eval()

Loaded configs/tsp10.json:
{'expt_name': 'tsp10', 'gpu_id': '0', 'train_filepath': './data/tsp10_train_concorde.txt', 'val_filepath': './data/tsp10_val_concorde.txt', 'test_filepath': './data/tsp10_test_concorde.txt', 'num_nodes': 14, 'num_neighbors': -1, 'node_dim': 2, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 30, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
    (obj): Sequential(
      (L0): Linear(in_features=128, out_features=128, bias=True)
      (A0): ReLU()
    )
    (linear_obj): Linear(in_features=128, out_features=1, bias=True)
    (m_obj): Sigmoid()
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=4, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=3, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): N

### Generate instances

In [7]:
num_instances = 100

In [8]:
# param_list = []
# scenario_tree_list = []
# for _ in range(num_instances):    
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
# #     network_problem = ndp.NetworkDesign(param)
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=lb, 
#                                   ub=ub, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
#     param_list.append(param)
#     scenario_tree_list.append(scenario_tree)
# df=open('test_data.txt', 'wb')
# pickle.dump(param_list, df, -1)
# pickle.dump(scenario_tree_list, df, -1)
# df.close() 

In [9]:
# # dependeence between context and scenarios
# param_list = []
# scenario_tree_list = []
# for _ in range(num_instances):    
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
#     ran_s = param['opening_cost'].mean()
#     ran_s = (ran_s-3)/8*15+5  
#     ran_l = param['shipping_cost'].copy()
#     ran_l[ran_l>999] = 0   
#     ran_l = ran_l.mean()
#     ran_l = (ran_l-5)/6*15+5   
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=round(ran_s)-5, 
#                                   ub=round(ran_l)+7, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
#     param_list.append(param)
#     scenario_tree_list.append(scenario_tree)
# df=open('test_data.txt', 'wb')
# pickle.dump(param_list, df, -1)
# pickle.dump(scenario_tree_list, df, -1)
# df.close()    

In [10]:
dfr=open('test_data.txt', 'rb')
param_list = pickle.load(dfr)
scenario_tree_list = pickle.load(dfr)

In [ ]:
#Define number of clusters
n_c = 2

our_objs = []
our_objs_objs = []
km_objs = []
optim = []

our_objs_t = []
our_objs_objs_t = []
km_objs_t = []
optim_t = []
for i in range(num_instances):
    # Instantiate Problem
#     param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
    param = param_list[i]
    network_problem = ndp.NetworkDesign(param)
#     network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
#                                   n_commodities=n_origins*n_destinations, 
#                                   distribution=distribution,
#                                   lb=lb, 
#                                   ub=ub, 
#                                   corr=corr)   
#     scenario_tree = network_uncertainty.get_scenario_tree()
    scenario_tree = scenario_tree_list[i]    
    
    ############################################################ 1. Clustering with nets
    with torch.no_grad():
        s_t = time.time()
        tensor_c = torch.from_numpy(np.array([param['opening_cost'],param['shipping_cost'],param['capacity']]))
        tensor_c = tensor_c.unsqueeze(0).repeat(n_scenarios, 1, 1, 1)
        ### notice! 0
        tensor_c[tensor_c>999] = 0
        tensor_c[tensor_c>99999] = 0
        ### notice! 20
        tensor_x = torch.from_numpy(scenario_tree.to_numpy()/20)
        ### 10 can be changed to any value > 1
        tensor_x = tensor_x.unsqueeze(1).repeat(1, 10, 1)
        x, y = tensor_x.to(device).float(), tensor_c.to(device).float() 
    
#         recon_x, mean, log_var, z, target, _, lat = vae(x, y)
#         X = z.detach().cpu().numpy()
#         #X = lat.detach().cpu().numpy()  
#         kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
#         repres =  kmedoids.cluster_centers_
#         indices= []
#         for j in range(repres.shape[0]):
#             indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
#         unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
#         if kmedoids.predict(repres)[0]!=unique_elements[0]:
#             counts_elements=counts_elements[::-1]
#         print(counts_elements) 
#         if len(counts_elements)<n_c:
#             continue
#         clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
#                                                          n_rvar={'d': 4},
#                                                          weights=np.array(counts_elements/counts_elements.sum()))  
#         solution_clustering_nets = network_problem.solve(clustering_nets, verbose=0)
#         our_objs_t.append(time.time()-s_t)        
#         stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['y']})
#         solution_nets = network_problem.solve(scenario_tree, 
#                                              decision_process=stage_0_decisions_nets, verbose=0)
#         our_objs.append(solution_nets.objective_value)   
    
    ###################################################################### 2. Clustering with nets with objectives
        s_t = time.time()    
        recon_x, mean, log_var, z, target, objs, _, lat = vae_objs(x, y)
        X = torch.cat((z,objs),1).detach().cpu().numpy()
        #X = lat.detach().cpu().numpy()    
        kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
        repres =  kmedoids.cluster_centers_
        indices= []
        for j in range(repres.shape[0]):
            indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
        unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
        if kmedoids.predict(repres)[0]!=unique_elements[0]:
            counts_elements=counts_elements[::-1]
        print(counts_elements)
        if len(counts_elements)==1:
#             our_objs.pop()
            continue        
        clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
                                                         n_rvar={'d': 4},
                                                         weights=np.array(counts_elements/counts_elements.sum()))    
        solution_clustering_nets = network_problem.solve(clustering_nets, verbose=0)
        our_objs_objs_t.append(time.time()-s_t)
        stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['y']})
        solution_nets = network_problem.solve(scenario_tree, 
                                             decision_process=stage_0_decisions_nets, verbose=0)
        our_objs_objs.append(solution_nets.objective_value)       
    ############################################################################# 3. start k-medoids
#     s_t = time.time()    
#     KX = scenario_tree.to_numpy()
#     Kkmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(KX)
#     Krepres =  Kkmedoids.cluster_centers_
#     Kindices= []
#     for j in range(Krepres.shape[0]):
#         Kindices.append(np.where(np.all(KX==Krepres[j], axis=1))[0][0])
#     Kunique_elements, Kcounts_elements = np.unique(Kkmedoids.labels_, return_counts=True)  
#     if Kkmedoids.predict(Krepres)[0]!=Kunique_elements[0]:
#         Kcounts_elements=Kcounts_elements[::-1]
#     print(Kcounts_elements) 
#     if len(Kcounts_elements)<n_c:
#         our_objs.pop()
#         our_objs_objs.pop()
#         continue      
#     Kclustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(Kindices)], 
#                                                      n_rvar={'d': 4},
#                                                      weights=np.array(Kcounts_elements/Kcounts_elements.sum())) 
#     Ksolution_clustering_nets = network_problem.solve(Kclustering_nets, verbose=0)
#     km_objs_t.append(time.time()-s_t)    
#     Kstage_0_decisions_nets = Ksolution_clustering_nets.decision_process_fixed({0: ['y']})
#     Ksolution_nets = network_problem.solve(scenario_tree, 
#                                          decision_process=Kstage_0_decisions_nets, verbose=0)
#     km_objs.append(Ksolution_nets.objective_value)
    ############################################################################### 4. actual solutions
#     s_t = time.time()     
#     actual_solution = network_problem.solve(scenario_tree, verbose=0)
#     optim_t.append(time.time()-s_t)       
#     optim.append(actual_solution.objective_value)
#     print(actual_solution.objective_value)
    print(str(i+1)+" instance(s) solved")

[ 88 112]
1 instance(s) solved
[105  95]
2 instance(s) solved
[ 98 102]
3 instance(s) solved
[103  97]
4 instance(s) solved
[ 95 105]
5 instance(s) solved
[105  95]
6 instance(s) solved
[121  79]
7 instance(s) solved
[105  95]
8 instance(s) solved
[111  89]
9 instance(s) solved
[105  95]
10 instance(s) solved
[ 89 111]
11 instance(s) solved
[ 99 101]
12 instance(s) solved
[ 78 122]
13 instance(s) solved
[109  91]
14 instance(s) solved
[ 99 101]
15 instance(s) solved
[ 95 105]
16 instance(s) solved
[ 92 108]
17 instance(s) solved
[ 81 119]
18 instance(s) solved
[108  92]
19 instance(s) solved
[ 83 117]
20 instance(s) solved
[ 97 103]
21 instance(s) solved
[ 91 109]
22 instance(s) solved
[ 96 104]
23 instance(s) solved
[ 87 113]
24 instance(s) solved
[ 96 104]
25 instance(s) solved
[ 81 119]
26 instance(s) solved
[ 90 110]
27 instance(s) solved
[ 94 106]
28 instance(s) solved
[105  95]
29 instance(s) solved
[103  97]
30 instance(s) solved
[ 95 105]
31 instance(s) solved
[ 91 109]
32 inst

In [13]:
optim = np.array([630.315     , 642.49      , 592.435     , 669.555     , 625.42      , 629.775     , 629.275     , 676.52      ,
       549.05      , 590.57      , 607.095     , 681.085     , 610.01      , 678.85      , 639.19      , 615.695     ,
       602.86      , 589.78      , 572.86      , 630.185     , 561.88      , 607.18      , 637.33      , 677.28      ,
       611.995     , 608.05      , 606.72      , 603.365     , 624.4       , 648.12999999, 579.97      , 641.545     ,
       685.3       , 620.905     , 691.415     , 733.945     , 652.96      , 618.475     , 699.05      , 636.775     ,
       643.82      , 649.795     , 631.355     , 636.2       , 588.17      , 682.785     , 657.315     , 564.22      ,
       612.835     , 650.67      , 591.605     , 662.275     , 585.765     , 579.94      , 574.795     , 640.565     ,
       644.54      , 648.88      , 569.9       , 670.86      , 603.685     , 651.55      , 586.375     , 607.265     ,
       638.65      , 646.245     , 671.99      , 637.365     , 671.575     , 623.08      , 651.465     , 573.725     ,
       603.795     , 616.495     , 649.16      , 637.535     , 647.25      , 626.015     , 679.595     , 642.365     ,
       580.54      , 623.99      , 643.28      , 613.32      , 604.1       , 629.73      , 621.965     , 641.9       ,
       628.755     , 596.37      , 613.555     , 616.67      , 659.755     , 620.965     , 713.615     , 621.18      ,
       602.015     ,660.735      ,663.165     , 664.945     ])

In [23]:
((np.array(our_objs)-np.array(optim))/np.array(optim)).std()  ####1000 with gcn

0.004915160140131036

In [24]:
((np.array(km_objs)-np.array(optim))/np.array(optim)).std()   ####1000 

0.00699153019085951

In [14]:
((np.array(our_objs_objs)-np.array(optim))/np.array(optim)).std()   ####1000 with gcn

0.7533768838392986

In [31]:
np.array(our_objs).mean(), np.array(our_objs).std()

(665.0670000000001, 52.89543613898406)

In [32]:
np.array(km_objs).mean(), np.array(km_objs).std()

(673.0951666666667, 65.39667199079955)

In [15]:
np.array(our_objs_objs).mean(), np.array(our_objs_objs).std()

(941.8574999999998, 477.26012303433646)

In [34]:
np.array(optim).mean(), np.array(optim).std()

(643.3894999994882, 36.120129252829315)

In [35]:
np.array(our_objs_t).mean()

0.6350385586420695

In [36]:
np.array(km_objs_t).mean()

0.2970906337102254

In [16]:
np.array(our_objs_objs_t).mean()

0.539507019519806

In [38]:
np.array(optim_t).mean()

16.876073479652405

In [39]:
np.array(optim)

array([625.17 , 663.16 , 624.065, 610.99 , 675.415, 654.625, 738.94 , 578.665, 722.62 , 677.84 , 661.715, 659.44 ,
       631.26 , 614.055, 676.92 , 633.845, 612.525, 700.4  , 637.225, 659.13 , 635.915, 605.825, 639.975, 592.47 ,
       642.165, 660.785, 607.22 , 606.705, 617.925, 634.695])

### With objectives

In [5]:
from models_objs import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae_objs = VAE(
    encoder_layer_sizes=[4, 256],
    latent_size=2,
    decoder_layer_sizes=[256, 4],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/ob_saved_model_"+ str(1999) + ".pt"
# load parameters
vae_objs.load_state_dict(torch.load(PATH))
vae_objs.eval()

VAE(
  (encoder): Encoder(
    (MLP): Sequential(
      (L0): Linear(in_features=6, out_features=256, bias=True)
      (A0): ReLU()
    )
    (linear_means): Linear(in_features=256, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=256, out_features=2, bias=True)
    (obj): Sequential(
      (L0): Linear(in_features=256, out_features=128, bias=True)
      (A0): ReLU()
    )
    (linear_obj): Linear(in_features=128, out_features=1, bias=True)
    (m_obj): Sigmoid()
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=256, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=256, out_features=4, bias=True)
      (sigmoid): Sigmoid()
    )
  )
  (compress): Sequential(
    (L0): Linear(in_features=588, out_features=128, bias=True)
    (A0): ReLU()
  )
  (linear_compress): Linear(in_features=128, out_features=2, bias=True)
  (m): Sigmoid()
)

In [6]:
our_objs = []
km_objs = []
optim = []
for i in range(100):
    # Instantiate Problem
    param = ndp.generate_random_parameters(n_origins, n_destinations, n_intermediates)
    network_problem = ndp.NetworkDesign(param)
    network_uncertainty = ndu.Demands(n_scenarios=n_scenarios,
                                  n_commodities=n_origins*n_destinations, 
                                  distribution=distribution,
                                  lb=lb, 
                                  ub=ub, 
                                  corr=corr)   
    scenario_tree = network_uncertainty.get_scenario_tree()
    # 1. Clustering with nets
    tensor_c = torch.from_numpy(np.array([param['opening_cost'],param['shipping_cost'],param['capacity']]))
    tensor_c = tensor_c.unsqueeze(0).repeat(200, 1, 1, 1)
    ### notice! 0
    tensor_c[tensor_c>999] = 0
    tensor_c[tensor_c>99999] = 0
    ### notice! 20
    tensor_x = torch.from_numpy(scenario_tree.to_numpy()/20)
    ### 10 can be changed to any value > 1
    tensor_x = tensor_x.unsqueeze(1).repeat(1, 10, 1)
    x, y = tensor_x.to(device).float(), tensor_c.to(device).float() 
    recon_x, mean, log_var, z, target, objs, _, lat = vae_objs(x, y)
    #X = torch.cat((z,objs),1).detach().cpu().numpy()
    X = lat.detach().cpu().numpy()    
    kmedoids = KMedoids(n_clusters=2, random_state=0).fit(X)
    repres =  kmedoids.cluster_centers_
    indices= []
    for j in range(repres.shape[0]):
        indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
    unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
    if kmedoids.predict(repres)[0]!=unique_elements[0]:
        counts_elements=counts_elements[::-1]
    clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
                                                     n_rvar={'d': 4},
                                                     weights=np.array(counts_elements/counts_elements.sum()))    
    solution_clustering_nets = network_problem.solve(clustering_nets, verbose=0)
    stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['y']})
    stage_0_decisions_nets
    solution_nets = network_problem.solve(scenario_tree, 
                                         decision_process=stage_0_decisions_nets, verbose=0)
    our_objs.append(solution_nets.objective_value)   
    #### start k-medoids
    KX = scenario_tree.to_numpy()
    Kkmedoids = KMedoids(n_clusters=2, random_state=0).fit(KX)
    Krepres =  Kkmedoids.cluster_centers_
    Kindices= []
    for j in range(Krepres.shape[0]):
        Kindices.append(np.where(np.all(KX==Krepres[j], axis=1))[0][0])
    Kunique_elements, Kcounts_elements = np.unique(Kkmedoids.labels_, return_counts=True)  
    if Kkmedoids.predict(Krepres)[0]!=Kunique_elements[0]:
        Kcounts_elements=Kcounts_elements[::-1]
    Kclustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(Kindices)], 
                                                     n_rvar={'d': 4},
                                                     weights=np.array(Kcounts_elements/Kcounts_elements.sum())) 
    Ksolution_clustering_nets = network_problem.solve(Kclustering_nets, verbose=0)
    Kstage_0_decisions_nets = Ksolution_clustering_nets.decision_process_fixed({0: ['y']})
    Ksolution_nets = network_problem.solve(scenario_tree, 
                                         decision_process=Kstage_0_decisions_nets, verbose=0)
    km_objs.append(Ksolution_nets.objective_value)
    ###
    actual_solution = network_problem.solve(scenario_tree, verbose=0)
    optim.append(actual_solution.objective_value)
    print(str(i+1)+"instance(s) solved")

1instance(s) solved
2instance(s) solved
3instance(s) solved
4instance(s) solved
5instance(s) solved
6instance(s) solved
7instance(s) solved
8instance(s) solved
9instance(s) solved
10instance(s) solved
11instance(s) solved
12instance(s) solved
13instance(s) solved
14instance(s) solved
15instance(s) solved
16instance(s) solved
17instance(s) solved
18instance(s) solved
19instance(s) solved
20instance(s) solved
21instance(s) solved
22instance(s) solved
23instance(s) solved
24instance(s) solved
25instance(s) solved
26instance(s) solved
27instance(s) solved
28instance(s) solved
29instance(s) solved
30instance(s) solved
31instance(s) solved
32instance(s) solved
33instance(s) solved
34instance(s) solved
35instance(s) solved
36instance(s) solved
37instance(s) solved
38instance(s) solved
39instance(s) solved
40instance(s) solved
41instance(s) solved
42instance(s) solved
43instance(s) solved
44instance(s) solved
45instance(s) solved
46instance(s) solved
47instance(s) solved
48instance(s) solved
4

In [7]:
((np.array(our_objs)-np.array(optim))/np.array(optim)).mean()

1.5678969861611967

In [8]:
((np.array(km_objs)-np.array(optim))/np.array(optim)).mean()

1.648452180809128

In [12]:
import math

In [19]:
torch.log(torch.tensor(0.8))

tensor(-0.2231)

In [16]:
math.log(1.2)

0.1823215567939546

In [ ]:
-0.2 0.2

In [20]:
0.0860 -0.2 -0.2+  0.1580

-0.15600000000000006

In [22]:
math.exp(-0.15600000000000006)

0.8555591903710185

In [23]:
0.0860 -0.3824 -0.4802+  0.158

-0.6185999999999999

In [24]:
math.exp(-0.6185999999999999)

0.538698087238962

In [12]:
0.0860 -0.3824 -0.4802+  0.158

-0.6185999999999999

In [16]:
np.random.poisson(1, 10)

array([1, 1, 2, 0, 0, 0, 1, 0, 2, 1])